In [13]:
from tensorflow.python.client import device_lib
print("\n\nShow Devices:\n\n"+str(device_lib.list_local_devices()))




Show Devices:

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5289309320769633738
]


In [7]:
import requests
import json
import csv
import io
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from keras import Sequential
from keras.layers import Dense, LSTM, Dropout, CuDNNLSTM

URL_dict = {"Volume" : "https://data.bitcoinity.org/export_data.csv?c=e&data_type=volume&r=hour&t=b&timespan=30d",
            "Price" : "https://data.bitcoinity.org/export_data.csv?c=e&currency=USD&data_type=price&r=hour&t=l&timespan=30d",
            "MkCap" : "https://data.bitcoinity.org/export_data.csv?currency=USD&data_type=market_cap&t=l&timespan=30d",
            "TradesPerMin" : "https://data.bitcoinity.org/export_data.csv?c=e&data_type=tradespm&r=hour&t=a&timespan=30d",
            "Volatility" : "https://data.bitcoinity.org/export_data.csv?c=e&currency=USD&data_type=volatility&f=m10&g=15&r=hour&st=log&t=l&timespan=30d",
            "BidAskSpread" : "https://data.bitcoinity.org/export_data.csv?c=e&currency=USD&data_type=spread&f=m10&r=hour&st=log&t=l&timespan=30d",
            "BidAskSum" : "https://data.bitcoinity.org/export_data.csv?bp=10&bu=c&currency=USD&data_type=bidask_sum&exchange=coinbase&r=hour&t=m&timespan=30d",
            }

for URL_Name, Link in URL_dict.items():
    r = requests.get(Link)
    r = r.content.decode('utf-8')
    c=pd.read_csv(io.StringIO(r))
    URL_dict[URL_Name]=c.iloc[-716:]
    print(f"Converting {URL_Name} CSV to a Pandas DataFrame")

Converting Volume CSV to a Pandas DataFrame
Converting Price CSV to a Pandas DataFrame
Converting MkCap CSV to a Pandas DataFrame
Converting TradesPerMin CSV to a Pandas DataFrame
Converting Volatility CSV to a Pandas DataFrame
Converting BidAskSpread CSV to a Pandas DataFrame
Converting BidAskSum CSV to a Pandas DataFrame


In [8]:

BidAskSpread_bitx = URL_dict.get("BidAskSpread").iloc[:,[1]]
BidAskSpread_bitfinex = URL_dict.get("BidAskSpread").iloc[:,[3]]
BidAskSpread_bitstamp = URL_dict.get("BidAskSpread").iloc[:,[4]]
BidAskSpread_cexio = URL_dict.get("BidAskSpread").iloc[:,[5]]
BidAskSpread_coinbase = URL_dict.get("BidAskSpread").iloc[:,[6]]
BidAskSpread_exmo = URL_dict.get("BidAskSpread").iloc[:,[7]]
BidAskSpread_gemini = URL_dict.get("BidAskSpread").iloc[:,[8]]
BidAskSpread_itbit = URL_dict.get("BidAskSpread").iloc[:,[9]]
BidAskSpread_kraken = URL_dict.get("BidAskSpread").iloc[:,[10]]

BidAskSum_asks = URL_dict.get("BidAskSum").iloc[:,[1]]
BidAskSum_bids = URL_dict.get("BidAskSum").iloc[:,[2]]
BidAskSum_price = URL_dict.get("BidAskSum").iloc[:,[3]]

Price_bitx = URL_dict.get("Price").iloc[:,[1]]
Price_bitfinex = URL_dict.get("Price").iloc[:,[3]]
Price_bitstamp = URL_dict.get("Price").iloc[:,[4]]
Price_cexio = URL_dict.get("Price").iloc[:,[5]]
Price_coinbase = URL_dict.get("Price").iloc[:,[6]]
Price_exmo = URL_dict.get("Price").iloc[:,[7]]
Price_gemini = URL_dict.get("Price").iloc[:,[8]]
Price_itbit = URL_dict.get("Price").iloc[:,[9]]
Price_kraken = URL_dict.get("Price").iloc[:,[10]]

TradesPerMin_bitx = URL_dict.get("TradesPerMin").iloc[:,[1]]
TradesPerMin_bitfinex = URL_dict.get("TradesPerMin").iloc[:,[2]]
TradesPerMin_bitflyer = URL_dict.get("TradesPerMin").iloc[:,[3]]
TradesPerMin_bithumb = URL_dict.get("TradesPerMin").iloc[:,[4]]
TradesPerMin_bitstamp = URL_dict.get("TradesPerMin").iloc[:,[5]]
TradesPerMin_coinbase = URL_dict.get("TradesPerMin").iloc[:,[6]]
TradesPerMin_exmo = URL_dict.get("TradesPerMin").iloc[:,[7]]
TradesPerMin_gemini = URL_dict.get("TradesPerMin").iloc[:,[8]]
TradesPerMin_kraken = URL_dict.get("TradesPerMin").iloc[:,[9]]
TradesPerMin_others = URL_dict.get("TradesPerMin").iloc[:,[10]]

Volatility_bitx = URL_dict.get("Volatility").iloc[:,[1]]
Volatility_bitfinex = URL_dict.get("Volatility").iloc[:,[2]]
Volatility_bitstamp = URL_dict.get("Volatility").iloc[:,[3]]
Volatility_cexio = URL_dict.get("Volatility").iloc[:,[4]]
Volatility_coinbase = URL_dict.get("Volatility").iloc[:,[5]]
Volatility_exmo = URL_dict.get("Volatility").iloc[:,[6]]
Volatility_gemini = URL_dict.get("Volatility").iloc[:,[7]]
Volatility_itbit = URL_dict.get("Volatility").iloc[:,[8]]
Volatility_kraken = URL_dict.get("Volatility").iloc[:,[9]]

Volume_bitx = URL_dict.get("Volume").iloc[:,[1]]
Volume_bitbay = URL_dict.get("Volume").iloc[:,[2]]
Volume_bitfinex = URL_dict.get("Volume").iloc[:,[3]]
Volume_bitflyer = URL_dict.get("Volume").iloc[:,[4]]
Volume_bithumb = URL_dict.get("Volume").iloc[:,[5]]
Volume_bitstamp = URL_dict.get("Volume").iloc[:,[6]]
Volume_coinbase = URL_dict.get("Volume").iloc[:,[7]]
Volume_gemini = URL_dict.get("Volume").iloc[:,[8]]
Volume_kraken = URL_dict.get("Volume").iloc[:,[9]]
Volume_others = URL_dict.get("Volume").iloc[:,[10]]

In [9]:

DataSets = [BidAskSpread_bitx,
            BidAskSpread_bitfinex,
            BidAskSpread_bitstamp,
            BidAskSpread_cexio,
            BidAskSpread_coinbase,
            BidAskSpread_exmo,
            BidAskSpread_gemini,
            BidAskSpread_itbit,
            BidAskSpread_kraken,
            BidAskSum_asks,
            BidAskSum_bids,
            BidAskSum_price,
            Price_bitx,
            Price_bitfinex,
            Price_bitstamp,
            Price_cexio,
            Price_coinbase,
            Price_exmo,
            Price_gemini,
            Price_itbit,
            Price_kraken,
            TradesPerMin_bitx,
            TradesPerMin_bitfinex,
            TradesPerMin_bitflyer,
            TradesPerMin_bithumb,
            TradesPerMin_bitstamp,
            TradesPerMin_coinbase,
            TradesPerMin_exmo,
            TradesPerMin_gemini,
            TradesPerMin_kraken,
            TradesPerMin_others,
            Volatility_bitx,
            Volatility_bitfinex,
            Volatility_bitstamp,
            Volatility_cexio,
            Volatility_coinbase,
            Volatility_exmo,
            Volatility_gemini,
            Volatility_itbit,
            Volatility_kraken,
            Volume_bitx,
            Volume_bitbay,
            Volume_bitfinex,
            Volume_bitflyer,
            Volume_bithumb,
            Volume_bitstamp,
            Volume_coinbase,
            Volume_gemini,
            Volume_kraken,
            Volume_others,
           ]

In [10]:
ConcatFrame = pd.concat(DataSets, axis=1, join='inner')
scaler = MinMaxScaler(feature_range=(0,1))
ConcatFrame = pd.DataFrame(scaler.fit_transform(ConcatFrame),columns = ConcatFrame.columns)
print(ConcatFrame)

X = []  
y = []  
for i in range(55, 715):  
    X.append(ConcatFrame.iloc[i-55:i, :])
    y.append(ConcatFrame.iloc[i, [0]])
    #print(f"iteration {i}")

X = np.stack(X)
y = np.stack(y)

print(X.shape)
print(y.shape)

        bit-x  bitfinex  bitstamp    cex.io  coinbase      exmo    gemini  \
0    0.971231  0.034766  0.131101  0.036232  0.104576  0.352001  0.080519   
1    0.979591  0.046405  0.208298  0.033079  0.069403  0.343277  0.044476   
2    0.984600  0.080052  0.232546  0.082276  0.077751  0.225727  0.090062   
3    0.990388  0.066494  0.304377  0.081020  0.069759  0.253359  0.094184   
4    0.992489  0.090930  0.413099  0.103831  0.091479  0.164911  0.092247   
5    0.986319  0.066394  0.333141  0.056036  0.054974  0.254172  0.111990   
6    0.977796  0.095146  0.220535  0.042738  0.078548  0.246306  0.118684   
7    0.977842  0.020423  0.175797  0.039610  0.122388  0.322824  0.134572   
8    0.978522  0.117483  0.148077  0.105881  0.223701  0.445254  0.114124   
9    0.979141  0.142473  0.292659  0.092459  0.186690  0.317363  0.091315   
10   0.983753  0.062734  0.280430  0.082506  0.013862  0.174256  0.069230   
11   0.987884  0.093585  0.316317  0.099814  0.048688  0.425117  0.061158   

(660, 55, 50)
(660, 1)


In [11]:
model = Sequential()
model.add(CuDNNLSTM(units=50, return_sequences= True, input_shape=(660,55,50)))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))
model.add(CuDNNLSTM(units=50))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()

model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=200, batch_size=16)

W0704 14:46:54.468112 4493743552 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0704 14:46:54.482606 4493743552 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.



ValueError: Input 0 is incompatible with layer cu_dnnlstm_1: expected ndim=3, found ndim=4